In [ ]:
import dask
import numpy as np
import sklearn
from matplotlib import pyplot as plt
from matplotlib import cm
import copy
import astropy
import hdbscan
import pandas as pd

from astropy.io import fits

In [ ]:
# hdul = fits.open('http://gal-03.sai.msu.ru/~vtoptun/photometry/rcsed_v2_clean.fits', memmap=astropy.io.fits.Conf.use_memmap.defaultvalue, lazy_load_hdus=True)
# hdul = fits.open('http://gal-03.sai.msu.ru/~vtoptun/redshift/rcsed_v2.fits')
hdul = fits.open('rcsed_v2_clean.fits', memmap=astropy.io.fits.Conf.use_memmap.defaultvalue, lazy_load_hdus=True)

In [ ]:
cols = hdul[1].columns

In [ ]:
data = hdul[1].data[:10000]

In [ ]:
hdul.close()
del hdul

In [ ]:
DATA = pd.DataFrame(np.array(data).byteswap().newbyteorder()) #[:100]

In [ ]:
del data

In [ ]:
DATA.info(verbose=True,null_counts=True)

In [ ]:
DATA = DATA.select_dtypes(exclude='object')

In [ ]:
DATA.info()

In [ ]:
data_float = DATA.select_dtypes(include=['float32', 'float64'])
data_int = DATA.select_dtypes(include=['int8','int16','int32','int64','uint8'])

converted_float = data_float.apply(pd.to_numeric,downcast='float')
converted_int = data_int.apply(pd.to_numeric,downcast='unsigned')

del data_float, data_int

In [ ]:
DATA = pd.concat([converted_float, converted_int],axis=1)

del converted_float,converted_int

In [ ]:
DATA.info()

In [ ]:
# df = pd.DataFrame(data = [[1,2,3],[1,3,3]])
# df.replace([3,2],np.nan)

In [ ]:
null_list = [-2147483648, '', '', '', '', '', -9223372036854775808, -32768, -2147483648, -32768, '', -2147483648, '', '', -32768, '', -2147483648, -2147483648, '', '', '', '', '', '', '', -2147483648, -2147483648, -2147483648, '', 255, 255, '', -32768, -32768, '', '', -2147483648, '', -2147483648, '', -32768, '', '', -32768, '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

In [ ]:
pd.Series(null_list).unique()

In [ ]:
DATA.replace([-2147483648, -9223372036854775808,-32768,255, -999999488.0,
 99.0,
 0.0,
 -99.0,
 9999.0,
 -999.0,
 0.05299999937415123,
 float('inf'),
 3.5,
 -9999.0,
 4.619999885559082,
 0.05000000074505806,
 70.0,
 4.0,
 179.96949768066406,
 0.06128999963402748,
 24.6346492767334], np.nan, inplace=True)

In [ ]:
DATA.dropna(axis=1,
    how='all',
    thresh=None,
    subset=None,
    inplace=True)

In [ ]:
DATA.info()

In [ ]:
DATA.shape

In [ ]:
# (~DATA.z_sdss.isna()).sum()

In [ ]:
DATA.info(verbose=True,null_counts=True)

In [ ]:
pd.set_option('max_columns', None)
DATA.head()

**Наблюдения:**
- в некоторых колонках,  в которых присутствуют все значния есть очень странные значения вроде "-2147483648" или "-32768" и другие

In [ ]:
#    Здесь мы создадим датафрейм с наиболее 
#       частотными значениями по каждому из столбцов

top_values = []

for col in DATA.columns[1:]:
    vc = DATA[col].value_counts(ascending=False).head(1)
    top_values.append([vc.name, vc.index[0], vc[vc.index[0]]])
    
anomaly = pd.DataFrame(data=top_values, columns = ['column_name', 'anomaly_value', 'value_counts'])

In [ ]:
# Здесь мы создаем колонку с количеством
#  ненулевых значений в каждом из столбцов таблицы
#     DATA, начиная с ra

not_nan = []

for col in DATA.columns[1:]:
    not_nan.append((~DATA[col].isna()).sum())
    
not_nan = pd.Series(not_nan)

In [ ]:
# Дополним получившийся датафрейм столбцом, показывающим,
# какую долю от длины всего столбца занимает каждое аномальное значение

import copy

top_values_data = copy.copy(anomaly)

top_values_data['percent_total'] = (top_values_data.value_counts / len(DATA)).round(3)
top_values_data['percent_not_nan'] = (top_values_data.value_counts / not_nan).round(3)

In [ ]:
# Выведем на экран получившйся датафрейм

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(top_values_data.sort_values(by='percent_not_nan', ascending=False))

### Анализ данных на основе таблицы top_values_data

Таблица **top_values_data** содержит в себе следующие столбцы:
- **column_name**   -   названия столбцов таблицы **DATA**

- **anomaly_value**   -   наиболее частотные значения столбцов таблицы **DATA**

- **value_counts**   -   количество раз, которое *anomaly_value* встречается в столбце таблицы **DATA**

- **percent_total**   -   доля *anomaly_value* от всего объема столбца таблицы **DATA**

- **percent_not_nan**   -   доля *anomaly_value* от non-null объема столбца таблицы **DATA**

В ячейке ниже показано, сколько столбцов таблицы **DATA** содержат более **90% и т.д.** аномальных значений от общего количества non-null значений столбца. Оказывается, что около трети столбцов (162) содержат таки аномалий больше, чем **10%**.

**! Нужно понять, какой порог считать допустимым.**

**! Остальные можно отбросить.**


In [ ]:
for num in np.arange(0.9,0.0,-0.05).round(2):
    print(num, '   ', len(top_values_data.query('percent_not_nan > @num')))

In [ ]:
#     Добавим столбец non_null_part, показываающий 
#     какую часть от величины столбца составляют ненулевые значения

top_values_data['non_null_part'] = (not_nan / len(DATA)).round(3)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(top_values_data.sort_values(by='non_null_part', ascending=True))

In [ ]:
# Выясним сколько столбцов таблицы DATA  заполнены меньше чем на num*100 %

for num in np.arange(0.9,0.0,-0.1).round(2):
    print(num, '   ', len(top_values_data.query('non_null_part < @num')))

In [ ]:
len(top_values_data.query('(non_null_part<0.01)&(percent_not_nan<0.5)'))

In [ ]:
# Выведем на экран получившйся датафрейм

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(top_values_data.sort_values(by='value_counts', ascending=False))

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(DATA.describe(include='object'))

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(DATA.describe())

In [ ]:
top_values_data[top_values_data['anomaly_value']==float('inf')]

In [ ]:
# anomaly_to_drop = list(top_values_data.anomaly_value.value_counts()
#                        [top_values_data.anomaly_value.value_counts(ascending=False)>1].index)

In [ ]:
# DATA.replace(anomaly_to_drop, np.nan, inplace=True)

In [ ]:
# list(top_values_data.anomaly_value.value_counts()
#                        [top_values_data.anomaly_value.value_counts(ascending=False)>1].index)

In [ ]:
DATA.info()

In [ ]:
# Найдем колонки, в которых аномальные значения
#     составляют не менее 90% от всех ненулевых значений столбца 

# percent_not_nan - доля anomaly_value от non-null объема столбца таблицы DATA
# non_null_part - доля ненулевых значений  от величины столбца 

columns_drop = top_values_data[(top_values_data['percent_not_nan']*top_values_data['non_null_part'] > 0.9)
                               |(top_values_data['non_null_part'] > 0.9)
                               |(top_values_data['anomaly_value']==float('inf'))].column_name 

In [ ]:
# Удалим колонки из DATA, в которых анамальные значения
#     составляют о не менее 90% от всех ненулевых значений столбца 

DATA.drop(labels=columns_drop, axis = 1, inplace=True)

Кажется, что мы очистили таблицу от всех столбцов, несущих мало информации, так как они содержали большое количество NaN или аномальных значений.

**Следующим нашим шагом** будет использование **IterativeImputer** чтобы заполнить оставшиеся пустые значения таблицы.

К новой заполненной таблице можно будет применить методы понижения разверности: VAE, t-SNE, PCA и др.

А далее уже кластеризировать.

In [ ]:
DATA.info()

In [ ]:
DATA = DATA.select_dtypes(include=['float32', 'float64']).apply(pd.to_numeric,downcast='float')

In [ ]:
DATA.info()

In [ ]:
DATA_columns = DATA.columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler #StandardScaler
nrmlzr = MinMaxScaler()
x = nrmlzr.fit_transform(DATA)
del DATA

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# imputer = IterativeImputer(max_iter=10, tol=3*x.std().max(),random_state=42, 
#                            estimator=ExtraTreesRegressor(n_estimators=10, random_state=0))

# imputer = IterativeImputer(max_iter=10, tol=3*x.std().max(),random_state=42, 
#                            estimator= DecisionTreeRegressor(max_features='sqrt', random_state=0))

imputer = IterativeImputer(max_iter=10, tol=3*x.std().max(),random_state=42, 
                           estimator=KNeighborsRegressor(n_neighbors=5, n_jobs=20))

x = imputer.fit_transform(x)

In [ ]:
import pandas as pd
y = pd.read_csv('rcsed_iGrID.csv')[:10000]

In [ ]:
sdss_indx = list(y[~y.iGrID.isna()].index)

In [ ]:
x.shape

In [ ]:
from sklearn.decomposition import TruncatedSVD

tsvd = TruncatedSVD(n_components=1, algorithm='randomized', n_iter=5, random_state=42)
x_new = tsvd.fit_transform(x)

In [ ]:
x.shape

In [ ]:
x_new.shape

In [ ]:
%%time
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, allow_single_cluster=True).fit(x_new)

In [ ]:
y = y.loc[sdss_indx,:].to_numpy().flatten()
# clusterer.labels_[sdss_indx]

In [ ]:
m1 = sklearn.metrics.homogeneity_completeness_v_measure(y,clusterer.labels_[sdss_indx])
m1

In [ ]:
# text_file = open("m1.txt", "w")
# n = text_file.write(str(m1))
# text_file.close()

In [ ]:
# y

In [ ]:
# pip install torch torchvision

In [ ]:
# DATA = pd.DataFrame(data=DATA_data, columns=DATA_columns)

In [ ]:
# DATA.info(verbose=True,null_counts=True)

Далее применим VAE  (https://github.com/dgradoboev/project_galaxies/blob/master/VAE.ipynb)

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt

# import sys
# sys.path

from sklearn.cluster import *
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import astropy
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy import constants as const
import hdbscan
from astropy.io import fits

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torchvision import datasets, transforms
from torch.autograd import Variable
# from torchvision.utils import save_image

In [ ]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)

        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h)
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return torch.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, x.shape[1]))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

In [ ]:
vae = VAE(x_dim=x.shape[1], h_dim1= int(x.shape[1]/2), h_dim2=int(x.shape[1]/4), z_dim=1)
if torch.cuda.is_available():
    vae.cuda()

optimizer = optim.Adam(vae.parameters())
train_loader = torch.utils.data.DataLoader(dataset=x, batch_size=int(len(x)/10), shuffle=True)

def loss_function(recon_x, x, mu, log_var):
    criterion = nn.MSELoss().cuda()
    BCE = criterion(recon_x, x.view(-1, x.shape[1]))
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx in range(400):
        #data = torch.tensor(x.sample(n=1000).astype(np.float32).values)
        data = torch.tensor(x[np.random.choice(range(x.shape[0]), 1000)].astype(np.float32))
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [ ]:
for epoch in range(1, 70):
    train(epoch)

In [ ]:
mu, log_var = vae.encoder(torch.tensor(x.astype(np.float32)))
embed = vae.sampling(mu, log_var).detach().numpy()

In [ ]:
hdbScan = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, allow_single_cluster=True).fit(embed)

In [ ]:
m2 = homogeneity_completeness_v_measure(y,hdbScan.labels_[sdss_indx])
m2

In [ ]:
# text_file = open("m2.txt", "w")
# m = text_file.write(str(m2))
# text_file.close()
